# Mount Drive



# Importing Libraries 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install texthero
!pip3 install transformers
!pip3 install tensorflow_addons


     |████████████████████████████████| 235 kB 7.6 MB/s 
     |████████████████████████████████| 1.5 MB 7.3 MB/s 
     |████████████████████████████████| 749 kB 48.9 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 4.0 MB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 71.4 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 3.5 MB/s 


In [ ]:
!pip install pyyaml==5.4.1

     |████████████████████████████████| 636 kB 5.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
#!pip install tweet-preprocessor

import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
nltk.download('wordnet')
import texthero as hero       # to be learned
import re
from texthero import stopwords

from nltk.corpus import wordnet #to be learned

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('english'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
    df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    #df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
import tensorflow_addons as tfa

# Reading Data


In [ ]:
def read_conll(in_file, lowercase=False, max_example=None):
    csv_list = []
    with open(in_file) as f:
        for line in f.readlines():
            csv=[]
            sp = line.strip().split('\t')
            if len(sp)>1:
               csv.append(sp[0])
               csv.append(sp[1])
            csv_list.append(csv)
    return pd.DataFrame(csv_list)

In [ ]:
lid_train = read_conll("/content/drive/MyDrive/PRISM ENG-HIN VIT/lid_spaeng/train.conll") #Your respective address 

## change data

In [ ]:
lid_train.columns=["word","label"]

In [ ]:
lid_dev_label = read_conll("/content/drive/MyDrive/PRISM ENG-HIN VIT/lid_spaeng/dev.conll")


In [ ]:
lid_dev_label.columns=["word","label"]

## ONE HOT ENCODING CLASS LABELS

In [ ]:
lid_train.label.unique()

array([None, 'other', 'lang1', 'lang2', 'ne', 'unk', 'ambiguous', 'mixed',
       'fw'], dtype=object)

In [ ]:
def one_hot(data):
  column_names = ["word","lang1","lang2","other","ne"]
  df = pd.DataFrame(columns = column_names)

  for i in range(len(data)):
    text_temp = data['word'].iloc[i]
    if data['label'].iloc[i] == "ne":
      df.loc[i] = [text_temp,0,0,0,1]
    elif data["label"].iloc[i] == "other" or data["label"].iloc[i] == "ambiguous" or data["label"].iloc[i] == "fw" or data["label"].iloc[i] == "unk":
      df.loc[i] = [text_temp,0,0,1,0]
    elif data["label"].iloc[i] == "lang2":
      df.loc[i] = [text_temp,0,1,0,0]
    elif data["label"].iloc[i] == "lang1":
      df.loc[i] = [text_temp,1,0,0,0]
    else:
      df.loc[i]=[text_temp,0,0,0,0]
  return df


In [ ]:
lid_dev=one_hot(lid_dev)

In [ ]:
lid_dev.to_csv("encoded_lid_dev_span.csv")

In [ ]:
lid_train=lid_train.dropna()

In [ ]:
lid_train

,word,label
1,11:11,other
2,.....,other
3,make,lang1
4,a,lang1
5,wish,lang1
...,...,...
295275,callata,lang2
295276,borrega,lang2
295277,pelos,lang2
295278,necios,lang2


In [ ]:
lid_train=one_hot(lid_train)

In [ ]:
lid_train.to_csv("encoded_lid_train_span.csv")

In [ ]:
lid_dev

,word,lang1,lang2,other,ne
0,NaN,0.0,0.0,0.0,0.0
1,@_easanti,0.0,0.0,1.0,0.0
2,@mememecaigo,0.0,0.0,1.0,0.0
3,#todossomoscarlosperez,0.0,0.0,0.0,0.0
4,#hashtaglargo,0.0,1.0,0.0,0.0
...,...,...,...,...,...
47050,NaN,0.0,0.0,0.0,0.0
47051,ABOTABOTABOTABOOOOOO,0.0,1.0,0.0,0.0
47052,!!!!!!,0.0,0.0,1.0,0.0
47053,PIPIPPIPI,0.0,0.0,1.0,0.0


In [ ]:
lid_dev

,Unnamed: 0,word,lang1,lang2,other,ne
1,1,@_easanti,0.0,0.0,1.0,0.0
2,2,@mememecaigo,0.0,0.0,1.0,0.0
3,3,#todossomoscarlosperez,0.0,0.0,0.0,0.0
4,4,#hashtaglargo,0.0,1.0,0.0,0.0
7,7,I,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
47047,47047,OLEEE,0.0,1.0,0.0,0.0
47048,47048,!!!,0.0,0.0,1.0,0.0
47051,47051,ABOTABOTABOTABOOOOOO,0.0,1.0,0.0,0.0
47052,47052,!!!!!!,0.0,0.0,1.0,0.0


In [ ]:
lid_dev=pd.read_csv("/content/encoded_lid_dev_span.csv")
lid_train=pd.read_csv("/content/encoded_lid_train_span.csv")

In [ ]:
import math

In [ ]:
lid_train=lid_train[(lid_train["word"].isnull()==False)]

In [ ]:
lid_train=lid_train.drop("Unnamed: 0",axis=1)

KeyError: ignored

In [ ]:
target_col= lid_train.columns[1:]
feature_col=lid_train.columns[0:1]

In [ ]:
lid_dev_label=lid_dev_label.dropna()

In [ ]:
lid_train

,word,label
1,11:11,other
2,.....,other
3,make,lang1
4,a,lang1
5,wish,lang1
...,...,...
295275,callata,lang2
295276,borrega,lang2
295277,pelos,lang2
295278,necios,lang2


#Preprocessing

In [ ]:
target_col,feature_col

(Index(['lang1', 'lang2', 'other', 'ne'], dtype='object'),
 Index(['word'], dtype='object'))

### preprocessing training and testing data

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
with tf.device('/GPU:0'): 
 lid_tr_prp= df_preprocessing(lid_train,feature_col[0])

In [ ]:
lid_dev

,Unnamed: 0,word,lang1,lang2,other,ne
0,0,NaN,0.0,0.0,0.0,0.0
1,1,easanti,0.0,0.0,1.0,0.0
2,2,mememecaigo,0.0,0.0,1.0,0.0
3,3,todossomoscarlosperez,0.0,0.0,0.0,0.0
4,4,hashtaglargo,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
47050,47050,NaN,0.0,0.0,0.0,0.0
47051,47051,abotabotabotaboooooo,0.0,1.0,0.0,0.0
47052,47052,,0.0,0.0,1.0,0.0
47053,47053,pipippipi,0.0,0.0,1.0,0.0


In [ ]:
 lid_dev_prp = df_preprocessing(lid_dev,feature_col[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

#Model 

In [ ]:
def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # # parameter declaration
  # step = tf.Variable(0, trainable=False)
  # schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
  # # lr and wd can be a function or a tensor
  # lr = learning_rate * schedule(step)
  # wd = lambda:lr * schedule(step)
  # optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  # optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.05)(x)
  # x= tf.keras.layers.Dense(128)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(4,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='categorical_crossentropy'),
                metrics=['accuracy'])
  return model

In [ ]:
pretrained_weights='bert-base-uncased'
max_len=256
epochs=1   # change epochs here
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
lid_tr_prp.shape

(253217, 5)

In [ ]:
lid_dev_prp.shape

(40391, 6)

In [ ]:
temp_tr=lid_tr_prp.iloc[:60000,:]
temp_tr.shape

(60000, 5)

In [ ]:
with tf.device('/GPU:0'):
  x_train= data_tokenization(temp_tr,feature_col[0],max_len,tokenizer) 

In [ ]:
y_train= temp_tr[target_col].values
y_train

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [ ]:
y_train.shape

(253217, 4)

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

Model selected: bert-base-uncased


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


15000/15000 [==============================] - 7012s 466ms/step - loss: 0.1994 - accuracy: 0.9392


In [ ]:
bert.save('/content/drive/MyDrive/distilbert_spanish_pred/my_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/distilbert_spanish_pred/my_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/distilbert_spanish_pred/my_model/assets


In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
print(x_train)

NameError: ignored

## tokenizing test data

In [ ]:
x_test= data_tokenization(lid_dev_prp,feature_col[0],max_len,tokenizer)
x_test

[array([[  101, 19413, 22341, ...,     0,     0,     0],
        [  101,  2033,  4168, ...,     0,     0,     0],
        [  101, 28681, 15094, ...,     0,     0,     0],
        ...,
        [  101, 11113, 17287, ...,     0,     0,     0],
        [  101,   102,     0, ...,     0,     0,     0],
        [  101, 28315, 11514, ...,     0,     0,     0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)]

In [ ]:
preds= bert.predict(x_test)

In [ ]:
preds.shape
predicted_test_labels=pd.DataFrame(preds)

In [ ]:
true_labels=lid_dev_label.label

In [ ]:
true_labels=true_labels.dropna()
true_labels=true_labels.replace(["unk","fw","ambiguous","mixed"],"other")

In [ ]:
true_labels.unique()

array(['other', 'lang2', 'lang1', 'ne'], dtype=object)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
true_labels=list(true_labels)

In [ ]:
len(true_labels)

40391

In [ ]:
predicted_test_labels.shape

(40391, 4)

In [ ]:
predicted_test_labels.columns=["lang1","lang2","other","ne"]

In [ ]:
pred_labels=[]
for i in range(predicted_test_labels.shape[0]):
  temp = max(predicted_test_labels['lang1'].iloc[i],predicted_test_labels['lang2'].iloc[i],predicted_test_labels['other'].iloc[i],predicted_test_labels['ne'].iloc[i])
  if temp == predicted_test_labels['lang1'].iloc[i]:
    pred_labels.append('lang1')
  elif temp == predicted_test_labels['lang2'].iloc[i]:
    pred_labels.append('lang2')
  elif temp == predicted_test_labels['other'].iloc[i]:
    pred_labels.append('other')
  elif temp == predicted_test_labels['ne'].iloc[i]:
    pred_labels.append('ne')

## performance on dev data 

In [ ]:
print(classification_report(true_labels,pred_labels))

              precision    recall  f1-score   support

       lang1       0.96      0.94      0.95     16712
       lang2       0.90      0.98      0.94     14955
          ne       0.70      0.47      0.56       815
       other       0.99      0.91      0.95      7909

    accuracy                           0.94     40391
   macro avg       0.89      0.82      0.85     40391
weighted avg       0.94      0.94      0.94     40391



### Multinomial Naive Bayes

In [ ]:
lid_train["label"].unique()

array(['other', 'lang1', 'lang2', 'ne', 'unk', 'ambiguous', 'mixed', 'fw'],
      dtype=object)

In [ ]:
lid_train=lid_train.replace("unk","other")
lid_train=lid_train.replace("ambiguous","other")
lid_train=lid_train.replace("mixed","other")
lid_train=lid_train.replace("fw","other")

lid_dev_label=lid_dev_label.replace("ambiguous","other")
lid_dev_label=lid_dev_label.replace("unk","other")
lid_dev_label=lid_dev_label.replace("mixed","other")
lid_dev_label=lid_dev_label.replace("fw","other")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(lid_train.word)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, lid_train.label)

In [ ]:
dev_counts=count_vect.transform(lid_dev_label.word)

In [ ]:
dev_vect=tfidf_transformer.transform(dev_counts)

In [ ]:
nb_pred=clf.predict(dev_vect)
print(classification_report(lid_dev_label.label,nb_pred))

              precision    recall  f1-score   support

       lang1       0.98      0.83      0.90     16712
       lang2       0.62      0.99      0.76     14955
          ne       1.00      0.16      0.28       815
       other       0.87      0.24      0.38      7909

    accuracy                           0.76     40391
   macro avg       0.87      0.56      0.58     40391
weighted avg       0.83      0.76      0.74     40391



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfmodel=RandomForestClassifier()
rfmodel.fit(X_train_counts,lid_train.label)
rfpred=rfmodel.predict(dev_counts)
print(classification_report(lid_dev_label.label,rfpred))

              precision    recall  f1-score   support

       lang1       0.98      0.83      0.90     16712
       lang2       0.97      0.85      0.91     14955
          ne       0.87      0.53      0.66       815
       other       0.63      0.99      0.77      7909

    accuracy                           0.87     40391
   macro avg       0.86      0.80      0.81     40391
weighted avg       0.91      0.87      0.87     40391



In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()